In [2]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate
# To run the training on TPU, you will need to uncomment the following line:
# !pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!apt install git-lfs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 11.2 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 5.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-

log to Git

In [3]:
!git config --global user.email "basile.plus-gourdon@ens-paris-saclay.fr"
!git config --global user.name "basileplus"

log to HF

In [4]:
from huggingface_hub import notebook_login

notebook_login()

Load dataset

In [77]:
from datasets import load_dataset

raw_datasets = load_dataset("stanfordnlp/coqa")

Take a look a the dataset

In [78]:
print(raw_datasets)

DatasetDict({
    train: Dataset({
        features: ['source', 'story', 'questions', 'answers'],
        num_rows: 7199
    })
    validation: Dataset({
        features: ['source', 'story', 'questions', 'answers'],
        num_rows: 500
    })
})


In [8]:
print(raw_datasets['train'].info.features)

{'source': Value(dtype='string', id=None), 'story': Value(dtype='string', id=None), 'questions': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'answers': Sequence(feature={'input_text': Value(dtype='string', id=None), 'answer_start': Value(dtype='int32', id=None), 'answer_end': Value(dtype='int32', id=None)}, length=-1, id=None)}


We format the dataset for it to fit the tutorial

In [174]:
def transform_example(batch):
    sources = []
    titles = []
    contexts = []
    questions = []
    answer_texts = []
    answer_starts = []
    answer_ends = []

    # Process each example in the batch
    for idx in range(len(batch["story"])):
        # num_answers = len(batch['answers'][idx]['input_text'])
        for num_question,question in enumerate(batch['questions'][idx]):
            # for a_idx in range(num_answers):
            sources.append(batch['source'][idx])
            titles.append(None)
            contexts.append(batch['story'][idx])
            questions.append(question)
            answer_texts.append(batch['answers'][idx]['input_text'][num_question])
            answer_starts.append(batch['answers'][idx]['answer_start'][num_question])
            answer_ends.append(batch['answers'][idx]['answer_end'][num_question])

    # Return a dictionary with the new structure
    return {
        'source': sources,
        'title': titles,
        'context': contexts,
        'question': questions,
        'answer_text': answer_texts,  # Changed to separate field
        'answer_start': answer_starts,  # Changed to separate field
        'answer_end': answer_ends
    }


In [175]:
from datasets import DatasetDict, concatenate_datasets

# Apply transformation and flatten the results
transformed_datasets = {}
for split, dataset in raw_datasets.items():
    # print(dataset.features)
    print("split : ", split)
    print("dataset : ", dataset)
    transformed_datasets[split] = dataset.map(transform_example, batched=True, remove_columns=dataset.column_names)
    # Flatten the list of lists into a single list of dicts

# Convert back to DatasetDict if necessary
transformed_dataset_dict = DatasetDict(transformed_datasets)



split :  train
dataset :  Dataset({
    features: ['source', 'story', 'questions', 'answers'],
    num_rows: 7199
})


Map:   0%|          | 0/7199 [00:00<?, ? examples/s]

split :  validation
dataset :  Dataset({
    features: ['source', 'story', 'questions', 'answers'],
    num_rows: 500
})


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [176]:
dataset = transformed_dataset_dict
dataset

DatasetDict({
    train: Dataset({
        features: ['source', 'title', 'context', 'question', 'answer_text', 'answer_start', 'answer_end'],
        num_rows: 108647
    })
    validation: Dataset({
        features: ['source', 'title', 'context', 'question', 'answer_text', 'answer_start', 'answer_end'],
        num_rows: 7983
    })
})

In [141]:
print(dataset["train"]["question"][0:25])
print(dataset["train"]["answer_text"][0:25])


['When was the Vat formally opened?', 'what is the library for?', 'for what subjects?', 'and?', 'what was started in 2014?', 'how do scholars divide the library?', 'how many?', 'what is the official name of the Vat?', 'where is it?', 'how many printed books does it contain?', 'when were the Secret Archives moved from the rest of the library?', 'how many items are in this secret collection?', 'Can anyone use this library?', 'what must be requested to view?', 'what must be requested in person or by mail?', 'of what books?', 'What is the Vat the library of?', 'How many books survived the Pre Lateran period?', 'what is the point of the project started in 2014?', 'what will this allow?', 'Where was the Auction held?', 'How much did they make?', 'How much did they expected?', 'WHo buy the Jackson Glove', 'Where was the buyer of the glove from?']
['It was formally established in 1475', 'research', 'history, and law', 'philosophy, science and theology', 'a  project', 'into periods', 'five', 'T

Create tokenizer

In [142]:
from transformers import AutoTokenizer

model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Always normalize data

In [177]:
questions = dataset["train"]["question"][0:3]
contexts = dataset["train"]["context"][0:3]
answers = dataset["train"]["answer_text"][0:3]

In [178]:
inputs = tokenizer(
    questions,
    contexts,
    max_length=100,
    truncation="only_second",
    stride=50,
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
)

for ids in inputs["input_ids"]:
  print(tokenizer.decode(ids))

[CLS] When was the Vat formally opened? [SEP] The Vatican Apostolic Library ( ), more commonly called the Vatican Library or simply the Vat, is the library of the Holy See, located in Vatican City. Formally established in 1475, although it is much older, it is one of the oldest libraries in the world and contains one of the most significant collections of historical texts. It has 75, 000 codices from throughout history, as well as 1. 1 million printed books, which [SEP]
[CLS] When was the Vat formally opened? [SEP] although it is much older, it is one of the oldest libraries in the world and contains one of the most significant collections of historical texts. It has 75, 000 codices from throughout history, as well as 1. 1 million printed books, which include some 8, 500 incunabula. The Vatican Library is a research library for history, law, philosophy, science and theology. The Vatican Library is open to anyone who can document their [SEP]
[CLS] When was the Vat formally opened? [SEP]

- max_length to set the maximum length (here 100)
- truncation="only_second" to truncate the context (which is in the second position) when the question with its context is too long
- stride to set the number of overlapping tokens between two successive chunks (here 50)
- return_overflowing_tokens=True to let the tokenizer know we want the overflowing tokens

Context has been split in parts because too long

In [147]:
print(f"The 3 examples gave {len(inputs['input_ids'])} features.")
print(f"Here is where each comes from: {inputs['overflow_to_sample_mapping']}.")

The 3 examples gave 18 features.
Here is where each comes from: [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2].


with ``inputs['overflow_to_sample_mapping']`` we keep track of from wich subsection of the truncated context we are talking



We then correct the inputs to create correct labels :

*   If the answer is in the sequence, labels are start and end indices of answer
*   Otherwise start=end=0



In [149]:
print(answers)

['It was formally established in 1475', 'research', 'history, and law']


offset mapping = beginning and end of token in real text

In [74]:
print(inputs["offset_mapping"])

[[(0, 0), (0, 4), (5, 8), (9, 12), (13, 14), (14, 16), (17, 25), (26, 32), (32, 33), (0, 0), (0, 3), (4, 11), (12, 21), (22, 29), (30, 31), (31, 32), (32, 33), (34, 38), (39, 47), (48, 54), (55, 58), (59, 66), (67, 74), (75, 77), (78, 84), (85, 88), (89, 90), (90, 92), (92, 93), (94, 96), (97, 100), (101, 108), (109, 111), (112, 115), (116, 120), (121, 124), (124, 125), (126, 133), (134, 136), (137, 144), (145, 149), (149, 150), (151, 155), (155, 159), (160, 171), (172, 174), (175, 178), (178, 179), (179, 180), (181, 189), (190, 192), (193, 195), (196, 200), (201, 206), (206, 207), (208, 210), (211, 213), (214, 217), (218, 220), (221, 224), (225, 231), (232, 241), (242, 244), (245, 248), (249, 254), (255, 258), (259, 267), (268, 271), (272, 274), (275, 278), (279, 283), (284, 295), (296, 307), (308, 310), (311, 321), (322, 327), (327, 328), (329, 331), (332, 335), (336, 338), (338, 339), (339, 342), (343, 345), (345, 350), (351, 355), (356, 366), (367, 374), (374, 375), (376, 378), (37

overflow_to_sample_mapping indique après découpage du context (car trop long) à quel context appartient le ième élément de la liste

In [76]:
print(f"Here is where each comes from: {inputs['overflow_to_sample_mapping']}.")

Here is where each comes from: [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3].


**feature** : input data which contains information

**ids** : id of the token

**.keys()** : allows to see every key of a dictionnary


In [151]:
inputs.keys()
# print("input_ids = ", inputs["input_ids"])
# print("token_type_ids = ",inputs["token_type_ids"])

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'overflow_to_sample_mapping'])

In [179]:
start_positions = []
end_positions = []

questions = dataset["train"]["question"][0:3]
contexts = dataset["train"]["context"][0:3]
answers_text = dataset["train"]["answer_text"][0:3]
answers_start = dataset["train"]["answer_start"][0:3]
answers_end = dataset["train"]["answer_end"][0:3]

for i, offset in enumerate(inputs["offset_mapping"]):
    sample_idx = inputs["overflow_to_sample_mapping"][i]
    answer_text = answers_text[sample_idx]
    start_char = answers_start[sample_idx]
    end_char = answers_end[sample_idx]
    print(f"sample idx : {sample_idx} / answer text : {answer_text} / start char {start_char} / end char : {end_char}")
    sequence_ids = inputs.sequence_ids(i)

    # Find the start and end of the context
    idx = 0
    while sequence_ids[idx] != 1:
        idx += 1
    context_start = idx
    while sequence_ids[idx] == 1:
        idx += 1
    context_end = idx - 1

    # If the answer is not fully inside the context, label is (0, 0)
    if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
        start_positions.append(0)
        end_positions.append(0)
    else:
        # Otherwise it's the start and end token positions
        idx = context_start
        while idx <= context_end and offset[idx][0] <= start_char:
            idx += 1
        start_positions.append(idx - 1)

        idx = context_end
        while idx >= context_start and offset[idx][1] >= end_char:
            idx -= 1
        end_positions.append(idx + 1)

print(start_positions)
print(end_positions)

sample idx : 0 / answer text : It was formally established in 1475 / start char 151 / end char : 179
sample idx : 0 / answer text : It was formally established in 1475 / start char 151 / end char : 179
sample idx : 0 / answer text : It was formally established in 1475 / start char 151 / end char : 179
sample idx : 0 / answer text : It was formally established in 1475 / start char 151 / end char : 179
sample idx : 0 / answer text : It was formally established in 1475 / start char 151 / end char : 179
sample idx : 0 / answer text : It was formally established in 1475 / start char 151 / end char : 179
sample idx : 1 / answer text : research / start char 454 / end char : 494
sample idx : 1 / answer text : research / start char 454 / end char : 494
sample idx : 1 / answer text : research / start char 454 / end char : 494
sample idx : 1 / answer text : research / start char 454 / end char : 494
sample idx : 1 / answer text : research / start char 454 / end char : 494
sample idx : 1 / answer 

Found indexes corresponds to tokens indexes

In [183]:
idx = 0
sample_idx = inputs["overflow_to_sample_mapping"][idx]
answer = answers[sample_idx]

start = start_positions[idx]
end = end_positions[idx]
labeled_answer = tokenizer.decode(inputs["input_ids"][idx][start : end + 1])

print(f"Theoretical answer: {answer}, labels give: {labeled_answer}")

Theoretical answer: It was formally established in 1475, labels give: Formally established in 1475


Now let's wrapp it up in a function to apply it to the entire dataset

In [223]:
max_length = 384
stride = 128

def preprocess_training_examples(examples):
    questions = examples["question"]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )
    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    start_positions = []
    end_positions = []

    answers_text = examples["answer_text"]
    answers_start = examples["answer_start"]
    answers_end = examples["answer_end"]

    for i, offset in enumerate(offset_mapping):
      sample_idx = sample_map[i]
      start_char = answers_start[sample_idx]
      end_char = answers_end[sample_idx]
      sequence_ids = inputs.sequence_ids(i)

      # Find the start and end of the context
      idx = 0
      while sequence_ids[idx] != 1:
          idx += 1
      context_start = idx
      while sequence_ids[idx] == 1:
          idx += 1
      context_end = idx - 1

      # If the answer is not fully inside the context, label is (0, 0)
      if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
          start_positions.append(0)
          end_positions.append(0)
      else:
          # Otherwise it's the start and end token positions
          idx = context_start
          while idx <= context_end and offset[idx][0] <= start_char:
              idx += 1
          start_positions.append(idx - 1)

          idx = context_end
          while idx >= context_start and offset[idx][1] >= end_char:
              idx -= 1
          end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [227]:
train_dataset = dataset["train"].map(
    preprocess_training_examples,
    batched=True,
    remove_columns=dataset["train"].column_names,
)
len(dataset["train"]), len(train_dataset)

(108647, 151990)

## Preprocess validation data

We use the unused 'title' column to store from which example the generated feature comes

In [233]:
dataset['validation'] = dataset["validation"].rename_column('title','id')

In [234]:
def preprocess_validation_examples(examples):
    questions = examples["question"]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [235]:
validation_dataset = dataset["validation"].map(
    preprocess_validation_examples,
    batched=True,
    remove_columns = dataset["validation"].column_names,
)
len(dataset["validation"]), len(validation_dataset)

Map:   0%|          | 0/7983 [00:00<?, ? examples/s]

(7983, 10951)